In [1]:
import pandas as pd

import numpy as np

testdf = pd.read_csv('/home/data/test_new.csv')
testdf.info()
testdf.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58097 entries, 0 to 58096
Data columns (total 5 columns):
r_key         58097 non-null object
out_id        58097 non-null object
start_time    58097 non-null object
start_lat     58097 non-null float64
start_lon     58097 non-null float64
dtypes: float64(2), object(3)
memory usage: 2.2+ MB


,r_key,out_id,start_time,start_lat,start_lon
0,f6fa6b2a1fa250b3_SDK-XJ_eed80f24f496fc9a59f49e...,358962079107966,2018-09-01 15:54:12,43.943356,125.377718
1,a584728d1eb0fb5b_SDK-XJ_d60de6f0b8121b07383e80...,358962079111695,2018-09-01 13:16:11,43.886501,125.272971
2,7308d46abc5ec4d0_SDK-XJ_6dd3f0f118e9813c51ed22...,358962079120563,2018-09-01 18:08:36,43.867917,125.307853
3,7038defae966f837_SDK-XJ_c596c8fd60e6e6cad14518...,4A23256745CBA3B0,2018-09-01 17:39:09,32.843961,115.864957
4,d4324c1d7d32d377_SDK-XJ_181a752a238a3f720eb378...,851181601098201,2018-09-01 09:10:15,22.711616,113.318815


In [2]:

import math
import sys
import time
from datetime import date

import numpy as np
from sklearn.externals import joblib

import geohash
import argparse


In [17]:
model_file = 'conbinRF03_1106.pkl'
regressor = joblib.load(model_file)

In [18]:

def date_extractor(date_str,b,minutes_per_bin):
    # Takes a datetime object as a parameter
    # and extracts and returns a tuple of the form: (as per the data specification)
    # (time_cat, time_num, time_cos, time_sin, day_cat, day_num, day_cos, day_sin, weekend)
    # Split date string into list of date, time
    
    d = date_str.split()
    
    #safety check
    if len(d) != 2:
        return tuple([None,])
    
    # TIME (eg. for 16:56:20 and 15 mins per bin)
    #list of hour,min,sec (e.g. [16,56,20])
    time_list = [int(t) for t in d[1].split(':')]
    
    #safety check
    if len(time_list) != 3:
        return tuple([None,])
    
    # calculate number of minute into the day (eg. 1016)
    num_minutes = time_list[0] * 60 + time_list[1]
    
    # Time of the start of the bin
    time_bin = num_minutes / minutes_per_bin     # eg. 1005
    hour_bin = num_minutes / 60                  # eg. 16
    min_bin = (time_bin * minutes_per_bin) % 60  # eg. 45
    
    #get time_cat
    hour_str = str(hour_bin) if hour_bin / 10 > 0 else "0" + str(hour_bin)  # eg. "16"
    min_str = str(min_bin) if min_bin / 10 > 0 else "0" + str(min_bin)      # eg. "45"
    time_cat = hour_str + ":" + min_str                                     # eg. "16:45"
    
    # Get a floating point representation of the center of the time bin
    time_num = (hour_bin*60 + min_bin + minutes_per_bin / 2.0)/(60*24)      # eg. 0.7065972222222222
    
    time_cos = math.cos(time_num * 2 * math.pi)
    time_sin = math.sin(time_num * 2 * math.pi)
    
    # DATE
    # Parse year, month, day
    date_list = d[0].split('-')
    d_obj = date(int(date_list[0]),int(date_list[1]),int(date_list[2]))
    day_to_str = {0: "Monday",
                  1: "Tuesday",
                  2: "Wednesday",
                  3: "Thursday",
                  4: "Friday",
                  5: "Saturday",
                  6: "Sunday"}
    day_of_week = d_obj.weekday()
    day_cat = day_to_str[day_of_week]
    day_num = (day_of_week + time_num)/7.0
    day_cos = math.cos(day_num * 2 * math.pi)
    day_sin = math.sin(day_num * 2 * math.pi)
    
    year = d_obj.year
    month = d_obj.month
    day = d_obj.day
    
    weekend = 0
    #check if it is the weekend
    if day_of_week in [5,6]:
        weekend = 1
       
    return (year, month, day, time_cat, time_num, time_cos, time_sin, day_cat, day_num, day_cos, day_sin, weekend)

def bucketed_location(lat, lon):
    location = geohash.encode(float(lat), float(lon), g)
    return geohash.decode(location)

In [19]:
resultDf = pd.DataFrame()
resultDf.loc[:,"r_key"] = testdf.loc[:,"r_key"]
resultDf.head()

,r_key
0,f6fa6b2a1fa250b3_SDK-XJ_eed80f24f496fc9a59f49e...
1,a584728d1eb0fb5b_SDK-XJ_d60de6f0b8121b07383e80...
2,7308d46abc5ec4d0_SDK-XJ_6dd3f0f118e9813c51ed22...
3,7038defae966f837_SDK-XJ_c596c8fd60e6e6cad14518...
4,d4324c1d7d32d377_SDK-XJ_181a752a238a3f720eb378...


In [20]:
len(testdf)
resultDf['end_lat'] = 0
resultDf['end_lon'] = 0
resultDf.head()


,r_key,end_lat,end_lon
0,f6fa6b2a1fa250b3_SDK-XJ_eed80f24f496fc9a59f49e...,0,0
1,a584728d1eb0fb5b_SDK-XJ_d60de6f0b8121b07383e80...,0,0
2,7308d46abc5ec4d0_SDK-XJ_6dd3f0f118e9813c51ed22...,0,0
3,7038defae966f837_SDK-XJ_c596c8fd60e6e6cad14518...,0,0
4,d4324c1d7d32d377_SDK-XJ_181a752a238a3f720eb378...,0,0


In [21]:
for i in range(len(testdf)):
    startLat = float(testdf.iloc[i,3])
    startLon = float(testdf.iloc[i,4])
    x_start = math.cos(startLat) * math.cos(startLon)
    y_start = math.cos(startLat) * math.sin(startLon) 
    z_start = math.sin(startLat) 
    zippedFeatures = date_extractor(testdf.iloc[i,2], 12, 15)
    parameters = np.array((zippedFeatures[8],zippedFeatures[1]/12, x_start, y_start,z_start)).reshape(1, -1)
    prediction = regressor.predict(parameters)
    resultDf.iloc[i,1] = prediction[0][0]
    resultDf.iloc[i,2] = prediction[0][1]
resultDf.head()

,r_key,end_lat,end_lon
0,f6fa6b2a1fa250b3_SDK-XJ_eed80f24f496fc9a59f49e...,43.914452,119.038322
1,a584728d1eb0fb5b_SDK-XJ_d60de6f0b8121b07383e80...,43.886620,120.825457
2,7308d46abc5ec4d0_SDK-XJ_6dd3f0f118e9813c51ed22...,43.979001,105.522776
3,7038defae966f837_SDK-XJ_c596c8fd60e6e6cad14518...,32.965117,109.141026
4,d4324c1d7d32d377_SDK-XJ_181a752a238a3f720eb378...,25.478207,112.999175


In [22]:
resultDf.to_csv('conbinRF03.csv',index=None)

In [ ]:
resultDf.info()

In [ ]:
resultDf.head

In [ ]:
testdf.head